In [1]:
from pathlib import Path

data = Path("./results-06-09/")

list(data.iterdir())

[PosixPath('results-06-09/20220904-2326'),
 PosixPath('results-06-09/20220901-1826'),
 PosixPath('results-06-09/20220902-1436'),
 PosixPath('results-06-09/.ipynb_checkpoints')]

In [2]:
import json
from collections import defaultdict

experiment_csvs = defaultdict(list)

for experiment in data.iterdir():
    if experiment.name.startswith("."): continue

    settings_path = experiment / "settings.json"
    settings = json.loads(settings_path.read_text())
    
    key = (settings["workers"], settings["stale"])
    experiment_csvs[key].extend([file for file in experiment.iterdir() if file.name.endswith(".csv")])

In [3]:
experiment_csvs

defaultdict(list,
            {(4,
              500): [PosixPath('results-06-09/20220904-2326/worker-0-stats.csv'), PosixPath('results-06-09/20220904-2326/worker-1-stats.csv'), PosixPath('results-06-09/20220904-2326/worker-3-stats.csv'), PosixPath('results-06-09/20220904-2326/worker-2-stats.csv')],
             (4,
              0): [PosixPath('results-06-09/20220901-1826/worker-0-stats.csv'), PosixPath('results-06-09/20220901-1826/worker-1-stats.csv'), PosixPath('results-06-09/20220901-1826/worker-3-stats.csv'), PosixPath('results-06-09/20220901-1826/worker-2-stats.csv')],
             (4,
              10): [PosixPath('results-06-09/20220902-1436/worker-0-stats.csv'), PosixPath('results-06-09/20220902-1436/worker-1-stats.csv'), PosixPath('results-06-09/20220902-1436/worker-3-stats.csv'), PosixPath('results-06-09/20220902-1436/worker-2-stats.csv')]})

In [4]:
import pandas as pd
dfs = {}

for experiment, csvs in experiment_csvs.items():
    dfs[experiment] = [pd.read_csv(csv) for csv in csvs]

In [5]:
dfs

{(4,
  500): [       Unnamed: 0    train_loss  train_acc  val_loss  val_acc     time_refs
  0        0.000000           NaN        NaN  2.305257   0.0897  1.662334e+09
  1        1.000000      2.248952   0.242188  2.256845   0.2261  1.662334e+09
  2        2.000000      2.202918   0.359375  2.196383   0.4030  1.662335e+09
  3        3.000000      2.133555   0.480469  2.125333   0.5326  1.662335e+09
  4        4.000000      2.069779   0.558594  2.060723   0.5912  1.662336e+09
  ..            ...           ...        ...       ...      ...           ...
  140    140.000000      0.411171   0.882812  0.320952   0.9088  1.662391e+09
  141    141.000000      0.442415   0.895833  0.321203   0.9090  1.662391e+09
  142    142.000000      0.269946   0.921875  0.319608   0.9091  1.662392e+09
  143    143.000000      0.310096   0.914062  0.319482   0.9088  1.662392e+09
  144  58157.892979  38676.755624        NaN       NaN      NaN           NaN
  
  [145 rows x 6 columns],        Unnamed: 0    tr

In [30]:
from matplotlib import pyplot as plt
ref = {}
labels = {0: "Sync", 10: "Serene", 500: "async"}
last_ax = None
col = "train_loss"
for experiment, experiment_data in dfs.items():
    initial_time = min(worker_data["time_refs"][0] for worker_data in experiment_data)
    final_time = max(worker_data["time_refs"].iloc[-2] for worker_data in experiment_data)
    max_dif = -1000
    columns = [df["time_refs"] for df in experiment_data]
    
    step = 200
    for step in range(50, 20000, 50):
        for i in range(int(initial_time) + 1, int(final_time) + 1, step):
            try:
                workers_clock = [df[((i - step) < df) & (df < (i + step))].idxmax() for df in columns]
                print(workers_clock)
            except:
                continue
            max_dif = max(max_dif, max(workers_clock) - min(workers_clock))
        print(f"Experiment {experiment} max diff: {max_dif} with step {step}")
        

[0, 0, 0, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[2, 2, 2, 2]
[5, 5, 5, 5]
[6, 6, 6, 6]
Experiment (4, 500) max diff: 0 with step 50
[0, 0, 0, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[2, 2, 2, 2]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 5, 5, 5]
[5, 5, 5, 5]
[6, 6, 6, 6]
[6, 6, 6, 6]
[7, 7, 7, 7]
[8, 8, 8, 8]
[9, 9, 9, 9]
[10, 10, 10, 10]
[11, 11, 11, 11]
[12, 12, 12, 12]
[13, 13, 13, 13]
[14, 14, 14, 14]
[15, 15, 15, 15]
[16, 16, 16, 16]
[17, 17, 17, 17]
[18, 18, 18, 18]
[19, 19, 19, 19]
[20, 20, 20, 20]
[30, 30, 30, 30]
[31, 31, 31, 31]
[32, 32, 32, 32]
[33, 33, 33, 33]
[34, 34, 34, 34]
[35, 35, 35, 35]
[36, 36, 36, 36]
[37, 37, 37, 37]
[38, 38, 38, 38]
Experiment (4, 500) max diff: 0 with step 100
[0, 0, 0, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[2, 2, 2, 2]
[2, 2, 2, 2]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 5, 5, 5]
[5, 5, 5, 5]
[6, 6, 6, 6]
[6, 6, 6, 6]
[7, 7, 7, 7]
[8, 8, 8, 8]
[9, 9, 9, 9]
[9, 9, 9, 9]
[10, 10, 10, 10]
[11, 11, 11, 11]
[12, 12, 12, 12]
[13, 13, 13, 13]
[14, 14, 14, 14]
[15, 15, 15, 15]
[16, 16, 16, 16]
[17, 17,